In [1]:
import math
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from model import TransformerModel

# Data processing and model compiling

In [2]:
#uj
in_seq_len = 20
out_seq_len = 5

def get_batch(source, i):
    #seq_len = min(bptt, len(source) - 1 - i)
    #modositott
    data = source[i:i+in_seq_len]
    target = source[i+in_seq_len:i+in_seq_len + out_seq_len]
    sos = torch.empty(1, target.shape[1], dtype = torch.int).fill_(vocab.stoi['<sos>'])
    #uj
    target = torch.cat((sos.to(device), target), dim = 0)
    return data, target

In [3]:
import io
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
counter = Counter()

#uj
counter.update(tokenizer('<sos>'))

for line in train_iter:
    counter.update(tokenizer(line))

vocab = Vocab(counter)

def data_process(raw_text_iter):
    data = [torch.tensor([vocab[token] for token in tokenizer(item)],
                       dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    
    #uj
    nseq = (data.size(0) - out_seq_len) // in_seq_len
    data = data.narrow(0, 0, nseq * in_seq_len + out_seq_len)
    
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [4]:
ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [5]:
len(train_data)

102485

In [6]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    #modositott
    trg_mask = model.generate_square_subsequent_mask(out_seq_len).to(device)
    for batch, i in enumerate(range(0, (train_data.size(0) - out_seq_len), in_seq_len)):
        data, targets = get_batch(train_data, i)
        
        trg_inp = targets[:-1,:]
        trg = targets[1:,:].reshape(-1)
        optimizer.zero_grad()
        output = model(data, trg_inp, src_mask = None, trg_mask = trg_mask)
        loss = criterion(output.view(-1, ntokens), trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // in_seq_len, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    trg_mask = model.generate_square_subsequent_mask(out_seq_len).to(device)
    with torch.no_grad():
        for i in range(0, (data_source.size(0) - out_seq_len), in_seq_len):
            data, targets = get_batch(data_source, i)
            trg_inp = targets[:-1,:]
            trg = targets[1:,:].reshape(-1)
            output = eval_model(data, trg_inp, src_mask = None, trg_mask = trg_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, trg).item()
    return total_loss / (len(data_source) - 1)

# Training

In [7]:
best_val_loss = float("inf")
epochs = 1 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

c:\users\kajud\appdata\local\programs\python\python38\lib\site-packages\torch\optim\lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |   200/ 5124 batches | lr 5.00 | ms/batch 15.98 | loss  8.65 | ppl  5733.12
| epoch   1 |   400/ 5124 batches | lr 5.00 | ms/batch 14.74 | loss  7.70 | ppl  2215.78
| epoch   1 |   600/ 5124 batches | lr 5.00 | ms/batch 14.49 | loss  7.47 | ppl  1757.15
| epoch   1 |   800/ 5124 batches | lr 5.00 | ms/batch 14.41 | loss  7.30 | ppl  1473.77
| epoch   1 |  1000/ 5124 batches | lr 5.00 | ms/batch 14.38 | loss  7.23 | ppl  1376.60
| epoch   1 |  1200/ 5124 batches | lr 5.00 | ms/batch 14.12 | loss  7.20 | ppl  1343.11
| epoch   1 |  1400/ 5124 batches | lr 5.00 | ms/batch 14.03 | loss  7.20 | ppl  1338.76
| epoch   1 |  1600/ 5124 batches | lr 5.00 | ms/batch 13.79 | loss  7.17 | ppl  1294.27
| epoch   1 |  1800/ 5124 batches | lr 5.00 | ms/batch 13.95 | loss  7.16 | ppl  1293.10
| epoch   1 |  2000/ 5124 batches | lr 5.00 | ms/batch 13.92 | loss  7.14 | ppl  1263.57
| epoch   1 |  2200/ 5124 batches | lr 5.00 | ms/batch 13.74 | loss  7.14 | ppl  1266.97
| epoch   1 |  2400/ 

# Evaluating

In [8]:
data, targets = get_batch(train_data, 10) # 1 szavankent vannak shiftelve a szekvenciak, target = kov input szekvencia

In [9]:
print("Input:")
' '.join([vocab.itos[i] for i in data[:,0].tolist()])

Input:


'chronicles ( japanese 戦場のヴァルキュリア3 , lit . valkyria of the battlefield 3 ) , commonly referred to as valkyria chronicles'

In [10]:
print("Target:")
' '.join([vocab.itos[i] for i in targets.view(out_seq_len + 1, batch_size)[:,0].tolist()])

Target:


'<sos> iii outside japan , is'

In [19]:
sentence = test_data[35:70,9].unsqueeze(0).cuda()
generated = sentence.transpose(0, 1)

In [20]:
print("Generating text with seed:")
' '.join([vocab.itos[i] for i in generated.transpose(0,1).tolist()[0]])

Generating text with seed:


'basketball player who represented australia in <unk> at the 2012 summer paralympics in london , coming 11th and 12th in her events . <unk> to wheelchair basketball , she made her debut with the national'

In [25]:
model.eval()
max_len = 10

src = model.emb_encoder(generated) * math.sqrt(model.ninp)
src = model.pos_encoder(src)
e_output = model.transformer_encoder(src, None)

outputs = torch.zeros(max_len).type_as(generated.data)
outputs[0] = torch.LongTensor([vocab.stoi['<sos>']])

for i in range(1, max_len):    
            
        trg_mask = model.generate_square_subsequent_mask(i).to(device)
        
        trg = model.emb_decoder(outputs[:i].unsqueeze(1)) * math.sqrt(model.ninp)
        trg = model.pos_encoder(trg)
        
        d_output = model.transformer_decoder(trg, e_output, trg_mask, None)
        out = model.decoder(d_output)
        out = torch.argmax(F.softmax(out, dim=-1), dim = -1).view(-1)[-1]
        outputs[i] = out
        

In [26]:
print("Generated text:")
' '.join([vocab.itos[i] for i in outputs.tolist()])

Generated text:


'<sos> the the the the the the the the the'